# Imports

In [1]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)

# Read and Make Classifiers Ready

In [2]:
df = pd.read_csv("newOutputs/comparison1.csv", index_col=0)

def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ["items","nodes","timesteps","arcs_ratio","values","PC1","PC2","PC3","PC4","PCweight1","PCweight2","PCweight3","PCweight4","PCAsAbove95","maxdegree/nodes","roots/nodes","leaves/nodes","longestpath/nodes","mean_constraint","mean_nr_similarities/nodes","mean_degree","nodestates","arcs","items/nodestates","items/arcs"]
df[columns] = df[columns].apply(normalize)

In [3]:
def combine(row):
    return row["algorithm"]+"_"+row["scorefunction"]

df = df[(df["algorithm"]!="TABU")|(df["scorefunction"]!="BDEU")]

df["algorithmMetric"] = df.apply(combine, axis=1)
df = df.drop(["algorithm","scorefunction"], axis=1)
df

,items,nodes,timesteps,arcs_ratio,values,fscore,tp,fp,tn,fn,structure_id,rep,PC1,PC2,PC3,PC4,PCweight1,PCweight2,PCweight3,PCweight4,PCAsAbove95,maxdegree/nodes,roots/nodes,leaves/nodes,longestpath/nodes,mean_constraint,mean_nr_similarities/nodes,mean_degree,PHI,nodestates,arcs,items/nodestates,items/arcs,hamming,items/PHI,compare,difference,algorithmMetric
0,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0,0.027363,0.022162,0.012024,1.967341e-34,0.444574,0.360072,0.195354,3.196361e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.666667,16.666667,0.400000,-0.400000,HC_BDEU
1,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0,0.030929,0.015200,0.013989,3.484487e-35,0.514469,0.252842,0.232689,5.796115e-34,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.666667,16.666667,0.400000,-0.400000,HC_AIC
2,0.01,0.50,0.333333,0.666666,0.5,0.500000,1,0,9,2,0,0,0.032252,0.022033,0.015422,1.501320e-34,0.462683,0.316078,0.221239,2.153749e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.833333,16.666667,0.400000,0.100000,HC_BIC
3,0.01,0.50,0.333333,0.666666,0.5,0.400000,1,1,8,2,0,0,0.034981,0.026677,0.007921,1.312431e-34,0.502751,0.383408,0.113841,1.886251e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.750000,16.666667,0.400000,0.000000,HC_K2
4,0.01,0.50,0.333333,0.666666,0.5,0.571429,2,2,7,1,0,0,0.022349,0.017244,0.012652,1.941603e-34,0.427763,0.330066,0.242172,3.716296e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.750000,16.666667,0.400000,0.171429,HC_L2L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94495,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,9,0.013596,0.008339,0.003139,1.247889e-03,0.503606,0.308887,0.116280,4.622290e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.933333,500.000000,0.307692,0.492308,MMHC_BDEU
94496,1.00,0.75,0.333333,0.666666,0.5,1.000000,5,0,25,0,9,9,0.012758,0.008644,0.003165,1.204312e-03,0.482216,0.326710,0.119641,4.551863e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,1.000000,500.000000,0.307692,0.692308,MMHC_BIC
94497,1.00,0.75,0.333333,0.666666,0.5,0.600000,3,2,23,2,9,9,0.012936,0.008565,0.003092,1.270001e-03,0.488332,0.323311,0.116705,4.794256e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.866667,500.000000,0.307692,0.292308,MMHC_K2
94498,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,9,0.012942,0.008717,0.003355,1.225521e-03,0.480217,0.323460,0.124495,4.547444e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.933333,500.000000,0.307692,0.492308,3OFF2_Constraint


In [4]:
dfhot = pd.get_dummies(df, columns=["algorithmMetric"], dtype=int)
dfhot

,items,nodes,timesteps,arcs_ratio,values,fscore,tp,fp,tn,fn,structure_id,rep,PC1,PC2,PC3,PC4,PCweight1,PCweight2,PCweight3,PCweight4,PCAsAbove95,maxdegree/nodes,roots/nodes,leaves/nodes,longestpath/nodes,mean_constraint,mean_nr_similarities/nodes,mean_degree,PHI,nodestates,arcs,items/nodestates,items/arcs,hamming,items/PHI,compare,difference,algorithmMetric_3OFF2_Constraint,algorithmMetric_HC_AIC,algorithmMetric_HC_BDEU,algorithmMetric_HC_BIC,algorithmMetric_HC_K2,algorithmMetric_HC_L2L,algorithmMetric_MIIC_Constraint,algorithmMetric_MMHC_BDEU,algorithmMetric_MMHC_BIC,algorithmMetric_MMHC_K2,algorithmMetric_TABU_AIC,algorithmMetric_TABU_BIC,algorithmMetric_TABU_K2,algorithmMetric_TABU_L2L
0,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0,0.027363,0.022162,0.012024,1.967341e-34,0.444574,0.360072,0.195354,3.196361e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.666667,16.666667,0.400000,-0.400000,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0.01,0.50,0.333333,0.666666,0.5,0.000000,0,1,8,3,0,0,0.030929,0.015200,0.013989,3.484487e-35,0.514469,0.252842,0.232689,5.796115e-34,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.666667,16.666667,0.400000,-0.400000,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0.01,0.50,0.333333,0.666666,0.5,0.500000,1,0,9,2,0,0,0.032252,0.022033,0.015422,1.501320e-34,0.462683,0.316078,0.221239,2.153749e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.833333,16.666667,0.400000,0.100000,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0.01,0.50,0.333333,0.666666,0.5,0.400000,1,1,8,2,0,0,0.034981,0.026677,0.007921,1.312431e-34,0.502751,0.383408,0.113841,1.886251e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.750000,16.666667,0.400000,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0.01,0.50,0.333333,0.666666,0.5,0.571429,2,2,7,1,0,0,0.022349,0.017244,0.012652,1.941603e-34,0.427763,0.330066,0.242172,3.716296e-33,0.300000,0.25,0.500000,0.500000,0.500000,0.500000,0.250000,0.125000,6,0.000244,0.250000,0.01,0.01,0.750000,16.666667,0.400000,0.171429,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94495,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,9,0.013596,0.008339,0.003139,1.247889e-03,0.503606,0.308887,0.116280,4.622290e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.933333,500.000000,0.307692,0.492308,0,0,0,0,0,0,0,1,0,0,0,0,0,0
94496,1.00,0.75,0.333333,0.666666,0.5,1.000000,5,0,25,0,9,9,0.012758,0.008644,0.003165,1.204312e-03,0.482216,0.326710,0.119641,4.551863e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,1.000000,500.000000,0.307692,0.692308,0,0,0,0,0,0,0,0,1,0,0,0,0,0
94497,1.00,0.75,0.333333,0.666666,0.5,0.600000,3,2,23,2,9,9,0.012936,0.008565,0.003092,1.270001e-03,0.488332,0.323311,0.116705,4.794256e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.866667,500.000000,0.307692,0.292308,0,0,0,0,0,0,0,0,0,1,0,0,0,0
94498,1.00,0.75,0.333333,0.666666,0.5,0.800000,4,1,24,1,9,9,0.012942,0.008717,0.003355,1.225521e-03,0.480217,0.323460,0.124495,4.547444e-02,0.266667,0.50,0.333333,0.333333,0.666667,0.527279,0.444444,0.222222,20,0.000977,0.416667,0.25,0.60,0.933333,500.000000,0.307692,0.492308,1,0,0,0,0,0,0,0,0,0,0,0,0,0


# Prediction

In [5]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [6]:
%%time
columns = ['items', 'arcs_ratio', 
   'PC1',
    'PCweight1', 
    'PCAsAbove95', 
    'maxdegree/nodes',
   'leaves/nodes', 'roots/nodes',  
   'mean_constraint', 
    'mean_degree',
   'items/nodestates', 'items/arcs',
    'PHI', 'items/PHI',
   'algorithmMetric_3OFF2_Constraint', 'algorithmMetric_HC_AIC',
   'algorithmMetric_HC_BDEU', 'algorithmMetric_HC_BIC',
   'algorithmMetric_HC_K2', 'algorithmMetric_HC_L2L',
   'algorithmMetric_MIIC_Constraint', 'algorithmMetric_MMHC_BDEU',
   'algorithmMetric_MMHC_BIC', 'algorithmMetric_MMHC_K2',
   'algorithmMetric_TABU_AIC', #'algorithmMetric_TABU_BDEU',
   'algorithmMetric_TABU_BIC', 'algorithmMetric_TABU_K2',
   'algorithmMetric_TABU_L2L']

X_train, X_test, y_train, y_test = train_test_split(dfhot[columns], dfhot["difference"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = pd.DataFrame(poly.fit_transform(X_train), columns=poly.get_feature_names_out(X_train.columns)), pd.DataFrame(poly.fit_transform(X_test), columns=poly.get_feature_names_out(X_test.columns))
regr = XGBRegressor(n_estimators=500, max_depth=14, eta=0.1, subsample=0.7, colsample_bytree=0.8)#, colsample_bynode=0.8)

regr.fit(X_train_poly, y_train)

print(regr.score(X_test_poly, y_test))

0.8527400990500159
CPU times: total: 50min 9s
Wall time: 5min 6s


# Feature Importance

In [7]:
def split(features):
    split1 = features.split(" ")
    if len(split1) > 1:
        return (split1[0], split1[1])
    
    split2 = features.split("^")
    if len(split2) > 1:
        return (split2[0], split2[0])
    
    return ('1',features)

features = pd.DataFrame({"features":regr.feature_names_in_, "importance":regr.feature_importances_}).sort_values("importance",ascending=True)
features["feature1"], features["feature2"] = zip(*features["features"].map(split))
features["importance"] *= 1 / max(features["importance"])

In [8]:
columns = ['1'] + list(X_train.columns)
combinations = pd.DataFrame()

for _, row in features.iterrows():
    if row["feature1"] == 2:
        print("A")
    combinations.loc[row["feature1"], row["feature2"]] = row["importance"]
    combinations.loc[row["feature2"], row["feature1"]] = row["importance"]
    
combinations.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)

,1,algorithmMetric_TABU_L2L,algorithmMetric_HC_AIC,algorithmMetric_HC_BDEU,algorithmMetric_HC_BIC,algorithmMetric_HC_K2,algorithmMetric_HC_L2L,algorithmMetric_MIIC_Constraint,algorithmMetric_MMHC_BDEU,algorithmMetric_MMHC_BIC,algorithmMetric_MMHC_K2,algorithmMetric_TABU_AIC,algorithmMetric_TABU_BIC,algorithmMetric_TABU_K2,items/PHI,PHI,algorithmMetric_3OFF2_Constraint,items/nodestates,items,PC1,PCAsAbove95,arcs_ratio,maxdegree/nodes,leaves/nodes,items/arcs,roots/nodes,PCweight1,mean_degree,mean_constraint
1,0.000000,0.041655,0.019283,0.022441,0.019205,0.019780,0.017601,0.030039,0.019252,0.031967,0.025314,0.048462,0.043728,0.055176,0.012404,0.023808,0.031321,0.001280,0.001823,0.003139,0.008107,0.003704,0.006079,0.010165,0.014514,0.008881,0.009490,0.024308,0.021853
algorithmMetric_HC_AIC,0.019283,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023141,0.035345,0.000000,0.042717,0.015190,0.029068,0.032688,0.040141,0.044418,0.048390,0.055232,0.031690,0.042634,0.033063,0.040651
algorithmMetric_TABU_L2L,0.041655,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057400,0.063022,0.000000,0.052387,0.037857,0.033538,0.048100,0.052334,0.067777,0.052956,0.061709,0.035661,0.047864,0.029719,0.035787
algorithmMetric_HC_BDEU,0.022441,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024599,0.024153,0.000000,0.017661,0.007361,0.028003,0.044486,0.042888,0.058483,0.040495,0.038600,0.043863,0.035302,0.057295,0.034445
algorithmMetric_HC_BIC,0.019205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019659,0.017759,0.000000,0.017245,0.016257,0.028682,0.049796,0.043082,0.057308,0.025985,0.035716,0.028731,0.028626,0.030582,0.019148
algorithmMetric_HC_K2,0.019780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047928,0.020351,0.000000,0.035939,0.017115,0.025396,0.060598,0.040158,0.037496,0.040180,0.033598,0.031028,0.033777,0.044865,0.027404
algorithmMetric_HC_L2L,0.017601,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055278,0.028776,0.000000,0.018390,0.028997,0.032696,0.029798,0.048067,0.056721,0.039559,0.063145,0.042090,0.030753,0.044890,0.035517
algorithmMetric_MIIC_Constraint,0.030039,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060045,0.036032,0.000000,0.027904,0.032211,0.055257,0.047530,0.077448,0.101360,0.061419,0.124971,0.095097,0.077505,0.041986,0.194257
algorithmMetric_MMHC_BDEU,0.019252,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033815,0.133628,0.000000,0.013172,0.015525,0.021419,0.044754,0.082461,0.058518,0.081956,0.026429,1.000000,0.059494,0.296729,0.298303
algorithmMetric_MMHC_BIC,0.031967,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016354,0.150971,0.000000,0.008911,0.014402,0.022200,0.068507,0.023516,0.167509,0.011551,0.030925,0.015328,0.035117,0.453378,0.158193
